### Install some libs

!pip install treelib

!pip3 install tree_sitter

### Import some libs

In [1]:
from treelib import Node, Tree
from tree_sitter import Language, Parser
import os
import numpy as np

### Initialize Tree-sitter go parser

In [2]:
Language.build_library(
  # Store the library in the `build` directory
  'build/my-languages.so',

  # Include one or more languages
  [
    'tree-sitter-go'
  ]
)

GO_LANGUAGE = Language('build/my-languages.so', 'go')

parser = Parser()
parser.set_language(GO_LANGUAGE)

## Step I : Find the Feature Toggles (FT)

```
$ egrep -R "=" kubernetes/pkg/features/kube_features.go | grep -v "//"
py> keys = [k.split('=')[0].split(' ')[0] for k in feats.replace('\t', '').split('\n')]
```

In [3]:
keywords = ['AppArmor',
 'DynamicKubeletConfig',
 'ExperimentalHostUserNamespaceDefaultingGate',
 'DevicePlugins',
 'HPAScaleToZero',
 'RotateKubeletServerCertificate',
 'LocalStorageCapacityIsolation',
 'ExpandPersistentVolumes',
 'ExpandInUsePersistentVolumes',
 'ExpandCSIVolumes',
 'EphemeralContainers',
 'QOSReserved',
 'CPUManager',
 'CPUCFSQuotaPeriod',
 'TopologyManager',
 'MemoryManager',
 'StorageObjectInUseProtection',
 'VolumeSubpath',
 'CSIInlineVolume',
 'CSIStorageCapacity',
 'DefaultPodTopologySpread',
 'GenericEphemeralVolume',
 'PreferNominatedNode',
 'RuntimeClass',
 'NetworkPolicyEndPort',
 'ProcMountType',
 'TTLAfterFinished',
 'IndexedJob',
 'JobTrackingWithFinalizers',
 'JobReadyPods',
 'KubeletPodResources',
 'CSIMigration',
 'CSIMigrationGCE',
 'InTreePluginGCEUnregister',
 'CSIMigrationAWS',
 'InTreePluginAWSUnregister',
 'CSIMigrationAzureDisk',
 'InTreePluginAzureDiskUnregister',
 'CSIMigrationAzureFile',
 'InTreePluginAzureFileUnregister',
 'CSIMigrationvSphere',
 'InTreePluginvSphereUnregister',
 'CSIMigrationOpenStack',
 'InTreePluginOpenStackUnregister',
 'CSIVolumeFSGroupPolicy',
 'ConfigurableFSGroupPolicy',
 'DelegateFSGroupToCSIDriver',
 'LocalStorageCapacityIsolationFSQuotaMonitoring',
 'NonPreemptingPriority',
 'PodOverhead',
 'IPv6DualStack',
 'EndpointSlice',
 'EndpointSliceProxying',
 'WindowsEndpointSliceProxying',
 'PodDisruptionBudget',
 'DaemonSetUpdateSurge',
 'ImmutableEphemeralVolumes',
 'HugePageStorageMediumSize',
 'DownwardAPIHugePages',
 'AnyVolumeDataSource',
 'SetHostnameAsFQDN',
 'WinOverlay',
 'WinDSR',
 'DisableAcceleratorUsageMetrics',
 'HPAContainerMetrics',
 'EndpointSliceTerminatingCondition',
 'ProxyTerminatingEndpoints',
 'EndpointSliceNodeName',
 'SizeMemoryBackedVolumes',
 'ExecProbeTimeout',
 'KubeletCredentialProviders',
 'DisableCloudProviders',
 'DisableKubeletCloudCredentialProviders',
 'CSIServiceAccountToken',
 'GracefulNodeShutdown',
 'ServiceLBNodePortControl',
 'MixedProtocolLBService',
 'VolumeCapacityPriority',
 'PodDeletionCost',
 'TopologyAwareHints',
 'ProbeTerminationGracePeriod',
 'NodeSwap',
 'PodAffinityNamespaceSelector',
 'ServiceLoadBalancerClass',
 'LogarithmicScaleDown',
 'IngressClassNamespacedParams',
 'ServiceInternalTrafficPolicy',
 'SuspendJob',
 'KubeletPodResourcesGetAllocatable',
 'NamespaceDefaultLabelName',
 'CSIVolumeHealth',
 'WindowsHostProcessContainers',
 'StatefulSetMinReadySeconds',
 'IdentifyPodOS',
 'ExpandedDNSConfig',
 'SeccompDefault',
 'PodSecurity',
 'ReadWriteOncePod',
 'CSRDuration',
 'KubeletInUserNamespace',
 'MemoryQoS',
 'CPUManagerPolicyOptions',
 'ControllerManagerLeaderMigration',
 'CPUManagerPolicyAlphaOptions',
 'CPUManagerPolicyBetaOptions',
 'JobMutableNodeSchedulingDirectives']

### List files with keywords

#### List all .go files

In [4]:
go_folders = [x[0] for x in os.walk("./kubernetes/")]

go_files = []
for dir_name in go_folders:
    files = [dir_name+"/"+k for k in os.listdir(dir_name) if k[len(k)-3:] ==".go"] # if k.endswith(".go")
    go_files.extend(files)

print("10 random .go files in this project :")
for k in range(10):
    print(go_files[np.random.randint(len(go_files))])

10 random .go files in this project :
./kubernetes/pkg/volume/awsebs/aws_ebs_test.go
./kubernetes/pkg/registry/core/podtemplate/strategy_test.go
./kubernetes/vendor/golang.org/x/sys/unix/ztypes_solaris_amd64.go
./kubernetes/staging/src/k8s.io/client-go/kubernetes/typed/apps/v1beta2/controllerrevision.go
./kubernetes/staging/src/k8s.io/apimachinery/pkg/util/rand/rand.go
./kubernetes/vendor/go.etcd.io/etcd/server/v3/etcdserver/api/v2http/httptypes/errors.go
./kubernetes/vendor/github.com/godbus/dbus/v5/default_handler.go
./kubernetes/staging/src/k8s.io/client-go/informers/internalinterfaces/factory_interfaces.go
./kubernetes/staging/src/k8s.io/client-go/applyconfigurations/core/v1/windowssecuritycontextoptions.go
./kubernetes/vendor/golang.org/x/net/http2/ascii.go


In [5]:
len(go_files)

15182

# List the interesting files, containing at least one keyword with the featureflag 

In [6]:
go_file_interests = []

for file in go_files:
    s = ""
    with open(file, "r") as f:
        s+=f.read()+"\n"
    kw_file = [k for k in keywords if k in s]
    if len(kw_file) > 0 and "featuregate" in s:
        print(file, "contains :")
        for kw in kw_file:
            print("->",kw)
        go_file_interests.append(file)
        print("\n")

./kubernetes/plugin/pkg/admission/priority/admission_test.go contains :
-> NonPreemptingPriority


./kubernetes/plugin/pkg/admission/priority/admission.go contains :
-> NonPreemptingPriority


./kubernetes/plugin/pkg/admission/noderestriction/admission.go contains :
-> ExpandPersistentVolumes


./kubernetes/plugin/pkg/admission/runtimeclass/admission_test.go contains :
-> RuntimeClass
-> PodOverhead


./kubernetes/plugin/pkg/admission/runtimeclass/admission.go contains :
-> RuntimeClass
-> PodOverhead


./kubernetes/plugin/pkg/admission/resourcequota/admission_test.go contains :
-> ExpandPersistentVolumes


./kubernetes/plugin/pkg/admission/security/podsecurity/admission_test.go contains :
-> PodSecurity


./kubernetes/plugin/pkg/admission/security/podsecurity/admission.go contains :
-> PodSecurity


./kubernetes/plugin/pkg/admission/security/podsecuritypolicy/admission_test.go contains :
-> AppArmor
-> CSIInlineVolume
-> PodSecurity


./kubernetes/plugin/pkg/auth/authorizer/node/node_

./kubernetes/pkg/volume/csi/csi_mounter_test.go contains :
-> CSIInlineVolume
-> CSIVolumeFSGroupPolicy
-> DelegateFSGroupToCSIDriver


./kubernetes/pkg/volume/csi/csi_plugin_test.go contains :
-> CSIInlineVolume
-> CSIVolumeFSGroupPolicy


./kubernetes/pkg/volume/csi/csi_attacher_test.go contains :
-> CSIInlineVolume
-> DelegateFSGroupToCSIDriver


./kubernetes/pkg/volume/csi/csi_client_test.go contains :
-> CSIVolumeHealth
-> ReadWriteOncePod


./kubernetes/pkg/volume/csi/csi_test.go contains :
-> CSIInlineVolume


./kubernetes/pkg/volume/csimigration/plugin_manager.go contains :
-> CSIMigration
-> CSIMigrationGCE
-> InTreePluginGCEUnregister
-> CSIMigrationAWS
-> InTreePluginAWSUnregister
-> CSIMigrationAzureDisk
-> InTreePluginAzureDiskUnregister
-> CSIMigrationAzureFile
-> InTreePluginAzureFileUnregister
-> CSIMigrationvSphere
-> InTreePluginvSphereUnregister
-> CSIMigrationOpenStack
-> InTreePluginOpenStackUnregister


./kubernetes/pkg/volume/csimigration/plugin_manager_test.go c

./kubernetes/pkg/controller/job/job_controller_test.go contains :
-> IndexedJob
-> JobTrackingWithFinalizers
-> JobReadyPods
-> SuspendJob


./kubernetes/pkg/security/podsecuritypolicy/provider_test.go contains :
-> AppArmor
-> EphemeralContainers
-> CSIInlineVolume
-> RuntimeClass
-> ProcMountType
-> PodSecurity


./kubernetes/pkg/api/pod/util_test.go contains :
-> AppArmor
-> LocalStorageCapacityIsolation
-> EphemeralContainers
-> ProcMountType
-> ConfigurableFSGroupPolicy
-> PodDeletionCost
-> ProbeTerminationGracePeriod
-> PodAffinityNamespaceSelector
-> IdentifyPodOS
-> ExpandedDNSConfig
-> PodSecurity


./kubernetes/pkg/api/v1/pod/util_test.go contains :
-> EphemeralContainers


./kubernetes/pkg/api/persistentvolume/util_test.go contains :
-> ExpandCSIVolumes


./kubernetes/pkg/api/persistentvolumeclaim/util_test.go contains :
-> AnyVolumeDataSource


./kubernetes/pkg/api/podsecuritypolicy/util_test.go contains :
-> ProcMountType
-> PodSecurity


./kubernetes/pkg/proxy/topology_t

In [7]:
len(go_file_interests)

148

## Step 3 - Search Feature Toggles dependencies in the AST

In [8]:
for gfi in go_file_interests:
    
    s = ""
    with open(gfi, "r") as f:
        s+=f.read()+"\n"

    print("File :            ", gfi, "\n")

    source = bytes(s, "utf8")
    ast = parser.parse(source)

    root_node = ast.root_node

    tree = Tree()

    root_node = ast.root_node

    type_nodes = dict()

    def get_name(node):
        return source[node.start_byte:node.end_byte].decode('utf8')

    def get_id(node):
        global type_nodes
        node_type = node.type
        if node_type not in type_nodes:
            type_nodes[node_type]=1
        else:
            type_nodes[node_type]+=1
        return node_type+str(type_nodes[node_type])

    def process(root_id, node):
        global tree
        node_id = get_id(node)
        node_name = get_name(node)
        for kw in keywords:
            if kw in node_name and node.type == 'if_statement':
                print("\n\n"+get_name(node)+"\n\n")
                for c in node.children:
                    if c.type == 'binary_expression' or c.type == 'unary_expression' or c.type == 'call_expression':
                        print("->",get_name(c))
        tree.create_node(node_name,
                         node_id,
                         parent = root_id)
        if len(node.children) != 0:
            for i in range(len(node.children)):
                process(node_id, node.children[i])

    tree.create_node("root", "root")
    for i in range(len(root_node.children)):
        process("root", root_node.children[i])
    print("\n")

    #tree.show()

File :             ./kubernetes/plugin/pkg/admission/priority/admission_test.go 



File :             ./kubernetes/plugin/pkg/admission/priority/admission.go 



File :             ./kubernetes/plugin/pkg/admission/noderestriction/admission.go 



File :             ./kubernetes/plugin/pkg/admission/runtimeclass/admission_test.go 



if featureInspection {
		relevantFeatures := map[featuregate.Feature]featuregate.FeatureSpec{
			features.PodOverhead: {Default: false},
		}
		fg := featuregate.NewFeatureGate()
		fg.Add(relevantFeatures)
		runtimeClass.InspectFeatureGates(fg)
	}




if addLister {
		informerFactory := informers.NewSharedInformerFactory(nil, controller.NoResyncPeriodFunc())
		runtimeClass.SetExternalKubeInformerFactory(informerFactory)
		if listerObject != nil {
			informerFactory.Node().V1().RuntimeClasses().Informer().GetStore().Add(listerObject)
		}
	}




if listerObject != nil {
			informerFactory.Node().V1().RuntimeClasses().Informer().GetStore().Add(listerObject)
	



File :             ./kubernetes/test/integration/scheduler/preemption_test.go 



File :             ./kubernetes/test/integration/scheduler/priorities_test.go 



File :             ./kubernetes/test/integration/quota/quota_test.go 



File :             ./kubernetes/test/integration/endpointslice/endpointsliceterminating_test.go 



File :             ./kubernetes/test/integration/pods/pods_test.go 



if _, err := setUpEphemeralContainers(client.CoreV1().Pods(ns.Name), pod, tc.original); err != nil {
			t.Errorf("%v: %v", tc.name, err)
		}


-> err != nil


if _, err := client.CoreV1().Pods(ns.Name).UpdateEphemeralContainers(context.TODO(), pod.Name, pod, metav1.UpdateOptions{}); tc.valid && err != nil {
			t.Errorf("%v: failed to update ephemeral containers: %v", tc.name, err)
		} else if !tc.valid && err == nil {
			t.Errorf("%v: unexpected allowed update to ephemeral containers", tc.name)
		}


-> tc.valid && err != nil


if _, err = client.CoreV1().Pods(ns.Name).UpdateEphemera



File :             ./kubernetes/pkg/scheduler/framework/plugins/volumebinding/binder_test.go 



if len(csiStorageCapacity) > 0 && csiStorageCapacity[0] {
		capacityCheck = &CapacityCheck{
			CSIDriverInformer:          csiDriverInformer,
			CSIStorageCapacityInformer: csiStorageCapacityInformer,
		}
	}


-> len(csiStorageCapacity) > 0 && csiStorageCapacity[0]


if scenario.needsCapacity && csiStorageCapacity &&
			csiDriver != nil && csiDriver.Spec.StorageCapacity != nil && *csiDriver.Spec.StorageCapacity {
			// Without CSIStorageCapacity objects, provisioning is blocked.
			expectedReasons = append(expectedReasons, ErrReasonNotEnoughSpace)
			expectedProvisions = nil
		}


-> scenario.needsCapacity && csiStorageCapacity &&
			csiDriver != nil && csiDriver.Spec.StorageCapacity != nil && *csiDriver.Spec.StorageCapacity


File :             ./kubernetes/pkg/quota/v1/evaluator/core/pods_test.go 



File :             ./kubernetes/pkg/quota/v1/evaluator/core/services_test.go 



File :



File :             ./kubernetes/pkg/registry/networking/networkpolicy/strategy_test.go 



File :             ./kubernetes/pkg/registry/networking/ingressclass/strategy_test.go 



File :             ./kubernetes/pkg/features/kube_features.go 



File :             ./kubernetes/pkg/controller/controller_utils_test.go 



File :             ./kubernetes/pkg/controller/certificates/signer/signer_test.go 



File :             ./kubernetes/pkg/controller/daemon/daemon_controller_test.go 



File :             ./kubernetes/pkg/controller/daemon/update_test.go 



File :             ./kubernetes/pkg/controller/endpointslice/reconciler_test.go 



if len(slices) != len(testCase.expectedEndpointPerSlice) {
				t.Fatalf("Expected %v EndpointSlice, got %d", len(testCase.expectedEndpointPerSlice), len(slices))
			}


-> len(slices) != len(testCase.expectedEndpointPerSlice)


if !strings.HasPrefix(slice.Name, testCase.service.Name) {
					t.Fatalf("Expected EndpointSlice name to start with %s, 



File :             ./kubernetes/pkg/controller/statefulset/stateful_set_status_updater_test.go 



File :             ./kubernetes/pkg/controller/statefulset/stateful_set_control.go 



if isRunningAndReady(pods[i]) {
			status.ReadyReplicas++
			// count the number of running and available replicas
			if utilfeature.DefaultFeatureGate.Enabled(features.StatefulSetMinReadySeconds) {
				if isRunningAndAvailable(pods[i], set.Spec.MinReadySeconds) {
					status.AvailableReplicas++
				}
			} else {
				// If the featuregate is not enabled, all the ready replicas should be considered as available replicas
				status.AvailableReplicas = status.ReadyReplicas
			}
		}


-> isRunningAndReady(pods[i])


if utilfeature.DefaultFeatureGate.Enabled(features.StatefulSetMinReadySeconds) {
				if isRunningAndAvailable(pods[i], set.Spec.MinReadySeconds) {
					status.AvailableReplicas++
				}
			} else {
				// If the featuregate is not enabled, all the ready replicas should be considered as availabl



if utilfeature.DefaultFeatureGate.Enabled(features.DynamicKubeletConfig) && len(s.DynamicConfigDir.Value()) > 0 &&
		kubeDeps.KubeletConfigController != nil && !standaloneMode && !s.RunOnce {
		if err := kubeDeps.KubeletConfigController.StartSync(kubeDeps.KubeClient, kubeDeps.EventClient, string(nodeName)); err != nil {
			return err
		}
	}


-> utilfeature.DefaultFeatureGate.Enabled(features.DynamicKubeletConfig) && len(s.DynamicConfigDir.Value()) > 0 &&
		kubeDeps.KubeletConfigController != nil && !standaloneMode && !s.RunOnce


if kubeServer.KubeletConfiguration.SeccompDefault && !utilfeature.DefaultFeatureGate.Enabled(features.SeccompDefault) {
		return fmt.Errorf("the SeccompDefault feature gate must be enabled in order to use the SeccompDefault configuration")
	}


-> kubeServer.KubeletConfiguration.SeccompDefault && !utilfeature.DefaultFeatureGate.Enabled(features.SeccompDefault)


if utilfeature.DefaultFeatureGate.Enabled(features.KubeletPodResources) {
		go k.ListenAndServeP